In [1]:

from utils import *


In [4]:




grammar = Grammar(start='SEQ')

for i in xrange(0,MAX+1):
    grammar.add_rule('INT', str(i), None, 1.0)
#grammar.add_rule('INT', str(INF), None, 5.0)

grammar.add_rule('INT', "len(%s)", ["SEQ"], 1.0)
grammar.add_rule('INT', "%s+%s", ["INT", "INT"], 2.0)
grammar.add_rule('INT', "%s-%s", ["INT", "INT"], 2.0)


#grammar.add_rule('SEQ', "'B'", None, 5.)
#grammar.add_rule('SEQ', "'A'", None, 5.)

grammar.add_rule('SEQ', 'from_seq', None, 10) 

#grammar.add_rule('SEQ', "'9'", None, 0.1)
#grammar.add_rule('SEQ', "'8'", None, 0.25)
#grammar.add_rule('SEQ', "'7'", None, 0.1)
#grammar.add_rule('SEQ', "'6'", None, 0.25)
#grammar.add_rule('SEQ', "'5'", None, 0.1)
#grammar.add_rule('SEQ', "'4'", None, 0.25)
#grammar.add_rule('SEQ', "'3'", None, 0.1)
#grammar.add_rule('SEQ', "'2'", None, 0.25)
#grammar.add_rule('SEQ', "'1'", None, 0.1)

#grammar.add_rule('SEQ', 'flip(%s)', ['INT'], 1.0) 
#grammar.add_rule('SEQ', 'all_rand(%s)', ['INT'], 1.0) 
#grammar.add_rule('SEQ', 'invert', ['SEQ'], 1.0) 
#grammar.add_rule('SEQ', "n_times(%s, %s)", ["SEQ", "INT"], 1.0)


grammar.add_rule('SEQ', "repeat(%s)", ["SEQ"], 1.0)
grammar.add_rule('SEQ', "alternate(%s, %s)", ["SEQ", "SEQ"], 1.0)

grammar.add_rule('SEQ', "append(%s, %s)", ["SEQ", "SEQ"], 1.0)
grammar.add_rule('SEQ', 'upto', ['SEQ', 'INT'], 1.0) 
grammar.add_rule('SEQ', 'after', ['SEQ', 'INT'], 1.0) 

grammar.add_rule('SEQ', "cut(%s, %s, %s)", ["SEQ", "INT", "INT"], 1.0)
grammar.add_rule('SEQ', 'increment', ['SEQ', 'SEQ'], 0.5) 
grammar.add_rule('SEQ', 'mirror', ['SEQ'], 1.) 
grammar.add_rule('SEQ', 'expand', ['SEQ'], 1.) 

grammar.add_rule('SEQ', 'next_dig', ['SEQ'], 1.) 
grammar.add_rule('SEQ', 'next_dig', ['SEQ'], 1.) 

grammar.add_rule('SEQ', 'index', ['SEQ', 'INT'], 1.) 

grammar.add_rule('SEQ', '((%s)(%s))',  ['FUNCDEF', 'FUNC'], 1.0)

# The function definition has a bound variable who can be applied as
# a function, whose arguments are an SEQ (set by the type of the FUNC above)
# and whose name is F, and who when applied to an SEQ returns an SEQ
# We'll also set bv_p here. Feel free to play with it and see what that does. 
grammar.add_rule('FUNCDEF', 'lambda', ['SEQ'], 1.0, bv_type='SEQ', bv_args=['SEQ'], bv_prefix='F')

# and we have to say what a FUNC is. It's a function (lambda) from an SEQ to an SEQ
grammar.add_rule('FUNC', 'lambda', ['SEQ'], 1.0, bv_type='SEQ')


class MyHypothesis(LOTHypothesis):
    
    

    def __init__(self, **kwargs):

        self.start_counts={}
        LOTHypothesis.__init__(self, grammar=grammar,
        maxnodes=400, display='lambda from_seq: %s', **kwargs)
        #LOTHypothesis.__init__(self, grammar=grammar,
        #maxnodes=400, display='lambda x: %s', **kwargs)


    def __call__(self, *args):
        out = LOTHypothesis.__call__(self, *args)
        return out
    

    def compute_single_likelihood(self, datum):
        alpha = datum.alpha
        true_val = datum.output.keys()[0][1]
        #generated = self(*datum.input)
        generated = self(datum.output.keys()[0][0])

       # generated = generated[:len(true_val)]
        #dist = (hamming_distance(generated, true_val[:len(generated)]) + 
                  #   len(true_val) - len(generated))

        p = max(0,1+abs(len(true_val) - len(generated))) * log(alpha)

        for i in xrange(0, min(len(generated), len(true_val))):
            v_g = generated[i]
            v_t = true_val[i]
            #if v_g not in "ABCD":
            #    if v_t == "B":
            #        p += log(float(v_g)/10.)
               # else:
                  #  p += log(1 - float(v_g)/10.)
           # else:
            if v_g == v_t:
                p += log(1.-alpha)
            else:
                p += log(alpha)


        return p

    


In [6]:


from LOTlib.SampleStream import *
from LOTlib.Inference.Samplers.MetropolisHastings import MHSampler
from LOTlib.DataAndObjects import FunctionData
from LOTlib.TopN import TopN
import copy
import re
from LOTlib.Eval import RecursionDepthException

        
            
# from model_helpers import *
#print repeat(lambda: flip(8))

ALPHA = 1e-8
STEPS = 4000
N_H = 50


stim = [("BBAA", "BBBAAA"), ("AABBB", "BBCCC"), ("ABCBA", "ABA"), 
        ("ABABB","CCBAB"), ("AAAAA", "BBBB"), ("AABBCCDD", "AABBCCDDEE") ][::-1]
rule_key = get_rule_key(grammar)

lst = []
lst_all = []
unique = set()
count = 0
for s in stim:

    data = [FunctionData(alpha=ALPHA,
         input=[""], output={s: len(s)})]

    h0 = MyHypothesis()
    seen = set()
    tn = TopN(N=N_H)


    for h in MHSampler(h0, data, 
            steps=STEPS, acceptance_temperature=2.): 
        out = h(s[0]) 
        if len(out) > len(s) and str(h) not in seen:
            seen.add(str(h))
            tn.add(h)
           # print h
           # print h(s[1])
            #print
            
    for h in tn.get_all(sorted=True):
        out = h(s[0]) 
        print h
      #  print s[1], out
        tup = (s[0], s[1], h(out), h(h(out)), h(h(h(out))))
        print tup
        print
        unique.add(tup)
    print "*" * 50
    
for u in unique:
    print u

 

lambda from_seq: append(from_seq, after(((lambda F3: next_dig(from_seq))(lambda y3: ((lambda F5: y3)(lambda y5: y5)))), 6))
('AABBCCDD', 'AABBCCDDEE', 'AABBCCDDEEEEFF', 'AABBCCDDEEEEFFEEFFFFGG', 'AABBCCDDEEEEFFEEFFFFGGEEFFFFGG')

lambda from_seq: append(append(from_seq, after(from_seq, 11)), after(next_dig(from_seq), 6))
('AABBCCDD', 'AABBCCDDEE', 'AABBCCDDEEEEFF', 'AABBCCDDEEEEFFEFFEEFFFFGG', 'AABBCCDDEEEEFFEFFEEFFFFGGEFFEF')

lambda from_seq: append(append(from_seq, after(from_seq, 15)), after(next_dig(from_seq), 6))
('AABBCCDD', 'AABBCCDDEE', 'AABBCCDDEEEEFF', 'AABBCCDDEEEEFFEEFFFFGG', 'AABBCCDDEEEEFFEEFFFFGGEFFFFGGE')

lambda from_seq: append(append(from_seq, after(from_seq, 13)), after(next_dig(from_seq), 6))
('AABBCCDD', 'AABBCCDDEE', 'AABBCCDDEEEEFF', 'AABBCCDDEEEEFFFEEFFFFGG', 'AABBCCDDEEEEFFFEEFFFFGGFFEEFFF')

lambda from_seq: append(append(from_seq, after(from_seq, 10)), after(next_dig(from_seq), 6))
('AABBCCDD', 'AABBCCDDEE', 'AABBCCDDEEEEFF', 'AABBCCDDEEEEFFEEFFEEFFFFGG', '

lambda from_seq: next_dig(((lambda F2: upto(from_seq, 4))(lambda y2: repeat(from_seq))))
('AAAAA', 'BBBB', 'CCCC', 'DDDD', 'EEEE')

lambda from_seq: next_dig(cut(from_seq, 1, 10+10))
('AAAAA', 'BBBB', 'CCC', 'DD', 'E')

lambda from_seq: next_dig(cut(from_seq, 1, 13+14))
('AAAAA', 'BBBB', 'CCC', 'DD', 'E')

lambda from_seq: next_dig(cut(from_seq, 1, 4+10))
('AAAAA', 'BBBB', 'CCC', 'DD', 'E')

lambda from_seq: next_dig(upto(((lambda F3: from_seq)(lambda y3: prev_dig(from_seq))), 4))
('AAAAA', 'BBBB', 'CCCC', 'DDDD', 'EEEE')

lambda from_seq: next_dig(cut(increment(from_seq, from_seq), 7, 11))
('AAAAA', 'BBBB', 'CCCC', 'DDDD', 'EEEE')

lambda from_seq: next_dig(((lambda F2: upto(F2(F2(from_seq)), 4))(lambda y2: from_seq)))
('AAAAA', 'BBBB', 'CCCC', 'DDDD', 'EEEE')

lambda from_seq: next_dig(((lambda F2: upto(F2(F2(from_seq)), 4))(lambda y2: y2)))
('AAAAA', 'BBBB', 'CCCC', 'DDDD', 'EEEE')

lambda from_seq: next_dig(upto(append(from_seq, next_dig(from_seq)), 4))
('AAAAA', 'BBBB', 'CCCC', 'D

lambda from_seq: upto(((lambda F2: F2(from_seq))(lambda y2: repeat(from_seq))), 3)
('ABCBA', 'ABA', 'ABC', 'ABC', 'ABC')

lambda from_seq: upto(((lambda F2: F2(from_seq))(lambda y2: repeat(y2))), 3)
('ABCBA', 'ABA', 'ABC', 'ABC', 'ABC')

lambda from_seq: after(((lambda F2: from_seq)(lambda y2: expand(y2))), 2)
('ABCBA', 'ABA', 'A', '', '')

lambda from_seq: upto(((lambda F2: from_seq)(lambda y2: next_dig(from_seq))), 3)
('ABCBA', 'ABA', 'ABC', 'ABC', 'ABC')

lambda from_seq: upto(((lambda F2: from_seq)(lambda y2: prev_dig(y2))), 3)
('ABCBA', 'ABA', 'ABC', 'ABC', 'ABC')

lambda from_seq: after(((lambda F2: from_seq)(lambda y2: expand(from_seq))), 2)
('ABCBA', 'ABA', 'A', '', '')

lambda from_seq: after(((lambda F2: from_seq)(lambda y2: prev_dig(from_seq))), 2)
('ABCBA', 'ABA', 'A', '', '')

lambda from_seq: after(((lambda F2: from_seq)(lambda y2: prev_dig(y2))), 2)
('ABCBA', 'ABA', 'A', '', '')

lambda from_seq: upto(((lambda F2: repeat(from_seq))(lambda y2: from_seq)), 3)
('ABCBA', 'AB

lambda from_seq: expand(((lambda F2: F2(F2(F2(F2(from_seq)))))(lambda y2: y2)))
('BBAA', 'BBBAAA', 'BBBBAAAA', 'BBBBBAAAAA', 'BBBBBBAAAAAA')

lambda from_seq: expand(((lambda F2: from_seq)(lambda y2: append(from_seq, from_seq))))
('BBAA', 'BBBAAA', 'BBBBAAAA', 'BBBBBAAAAA', 'BBBBBBAAAAAA')

lambda from_seq: expand(((lambda F2: F2(repeat(from_seq)))(lambda y2: from_seq)))
('BBAA', 'BBBAAA', 'BBBBAAAA', 'BBBBBAAAAA', 'BBBBBBAAAAAA')

lambda from_seq: expand(((lambda F2: from_seq)(lambda y2: append(from_seq, y2))))
('BBAA', 'BBBAAA', 'BBBBAAAA', 'BBBBBAAAAA', 'BBBBBBAAAAAA')

lambda from_seq: expand(((lambda F2: from_seq)(lambda y2: append(y2, y2))))
('BBAA', 'BBBAAA', 'BBBBAAAA', 'BBBBBAAAAA', 'BBBBBBAAAAAA')

lambda from_seq: ((lambda F1: expand(F1(F1(F1(from_seq)))))(lambda y1: from_seq))
('BBAA', 'BBBAAA', 'BBBBAAAA', 'BBBBBAAAAA', 'BBBBBBAAAAAA')

lambda from_seq: ((lambda F1: expand(F1(F1(F1(from_seq)))))(lambda y1: y1))
('BBAA', 'BBBAAA', 'BBBBAAAA', 'BBBBBAAAAA', 'BBBBBBAAAAAA')

